In [90]:
#imports pandas
import pandas as pd

import seaborn as sns 

import numpy as np

import matplotlib.pyplot as plt

from sklearn.neighbors import NearestNeighbors

#natural language toolkit




In [91]:
df1=pd.read_csv('/Users/all/Desktop/archive/RAW_recipes.csv')


In [92]:
ratings=pd.read_csv('/Users/all/Desktop/archive/RAW_interactions.csv')


In [93]:
ratings.user_id.value_counts()

424680        7671
37449         5603
383346        4628
169430        4076
128473        3917
              ... 
244983           1
139446           1
495446           1
113469           1
2001868099       1
Name: user_id, Length: 226570, dtype: int64

In [94]:
ratings.user_id.unique().shape

(226570,)

In [95]:
x = ratings['user_id'].value_counts() > 100
x[x].shape

(1368,)

In [96]:
y =  x[x].index
y

Int64Index([ 424680,   37449,  383346,  169430,  128473,   89831,   58104,
             133174,  199848,  305531,
            ...
             709541, 1182971, 1458294,  121684,  212417, 1020416,  804851,
             498487,  390058,    9580],
           dtype='int64', length=1368)

In [97]:
#takes users with more than 100 reviews
ratings= ratings[ratings['user_id'].isin(y)]

In [98]:
ratings.head()

,user_id,recipe_id,date,rating,review
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."
5,52282,120345,2005-05-21,4,very very sweet. after i waited the 2 days i b...
6,124416,120345,2011-08-06,0,"Just an observation, so I will not rate. I fo..."
8,76535,134728,2005-09-02,4,Very good!


In [99]:
ratings.drop('review',axis=1)

,user_id,recipe_id,date,rating
3,126440,85009,2010-02-27,5
4,57222,85009,2011-10-01,5
5,52282,120345,2005-05-21,4
6,124416,120345,2011-08-06,0
8,76535,134728,2005-09-02,4
...,...,...,...,...
1132353,126435,166739,2007-08-25,5
1132355,140132,82303,2010-10-01,5
1132356,199020,82303,2013-03-18,5
1132357,1122988,82303,2014-07-08,5


In [100]:
df1.columns
ratings.columns

Index(['user_id', 'recipe_id', 'date', 'rating', 'review'], dtype='object')

In [101]:
df1.rename(columns={
    'id':"recipe_id"})

,name,recipe_id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8
...,...,...,...,...,...,...,...,...,...,...,...,...
231632,zydeco soup,486161,60,227978,2012-08-29,"['ham', '60-minutes-or-less', 'time-to-make', ...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22
231633,zydeco spice mix,493372,5,1500678,2013-01-09,"['15-minutes-or-less', 'time-to-make', 'course...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13
231634,zydeco ya ya deviled eggs,308080,40,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8
231635,cookies by design cookies on a stick,298512,29,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10


In [102]:
data=pd.merge(df1,ratings,left_on='id',right_on='recipe_id')

In [103]:
data.head(2)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,user_id,recipe_id,date,rating,review
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,4470,137739,2006-02-18,5,I used an acorn squash and recipe#137681 Swee...
1,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,593927,137739,2010-08-21,5,This was a nice change. I used butternut squas...


In [104]:
data.shape

(510486, 17)

In [105]:
num_rating = data.groupby('name')['rating'].count().reset_index()

In [106]:
num_rating.head()

,name,rating
0,0 fat chunky watermelon salsa,1
1,0 point soup ww,7
2,0 point soup crock pot,4
3,007 martini,1
4,007 cocktail,1


In [107]:
num_rating.rename(columns={'rating':"number_of_rating"})

,name,number_of_rating
0,0 fat chunky watermelon salsa,1
1,0 point soup ww,7
2,0 point soup crock pot,4
3,007 martini,1
4,007 cocktail,1
...,...,...
173409,zydeco sauce,1
173410,zydeco shrimp wrap,1
173411,zydeco soup,6
173412,zydeco spice mix,1


In [108]:
#puts it into data
final_rating=data.merge(num_rating, on='name')

In [109]:
final_rating.head(2)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,user_id,recipe_id,date,rating_x,review,rating_y
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,4470,137739,2006-02-18,5,I used an acorn squash and recipe#137681 Swee...,3
1,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,593927,137739,2010-08-21,5,This was a nice change. I used butternut squas...,3


In [110]:
final_rating.rename({'rating_y': 'num_of_rating','rating_x': 'rating'}, axis=1,inplace=True)

In [111]:
final_rating.shape

(510486, 18)

In [112]:
final_rating=final_rating[final_rating['num_of_rating']>=25]

In [113]:
final_rating.head(3)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,user_id,recipe_id,date,rating,review,num_of_rating
163,never weep whipped cream,74805,5,87877,2003-11-01,"['15-minutes-or-less', 'time-to-make', 'course...","[276.3, 45.0, 2.0, 1.0, 3.0, 91.0, 0.0]",4,['whip all ingredients together until firm pea...,"i don't know where i got this, but it works. t...","['whipping cream', 'vanilla instant pudding mi...",4,215898,74805,2006-03-05,5,"Easy to make, delicious and holds up well!",27
164,never weep whipped cream,74805,5,87877,2003-11-01,"['15-minutes-or-less', 'time-to-make', 'course...","[276.3, 45.0, 2.0, 1.0, 3.0, 91.0, 0.0]",4,['whip all ingredients together until firm pea...,"i don't know where i got this, but it works. t...","['whipping cream', 'vanilla instant pudding mi...",4,177753,74805,2006-11-23,5,I've made this several times now and it always...,27
165,never weep whipped cream,74805,5,87877,2003-11-01,"['15-minutes-or-less', 'time-to-make', 'course...","[276.3, 45.0, 2.0, 1.0, 3.0, 91.0, 0.0]",4,['whip all ingredients together until firm pea...,"i don't know where i got this, but it works. t...","['whipping cream', 'vanilla instant pudding mi...",4,391048,74805,2006-12-11,4,It works! Needs some extra flavor though. Thanks!,27


In [114]:
final_rating.shape

(56909, 18)

In [116]:
final_rating.drop_duplicates(['user_id','name'],inplace=True)

In [117]:
final_rating.shape

(56883, 18)

In [120]:
##creates a pivot table

food_pivot = final_rating.pivot_table(columns='user_id', index = 'name', values = 'rating')

In [121]:
food_pivot

user_id,1533,1535,2310,2312,3288,4291,4439,4470,4740,5060,...,2206637,2324285,2399085,2549237,2597942,1800054678,1802849661,2000431901,2000498330,2001102678
name,,,,,,,,,,,,,,,,,,,,,
1 pan fudge cake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2 handed kitchen sink tomato sandwich,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20 minute creamy pesto chicken and bow ties,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24k carrots,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4 minute spicy garlic shrimp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zesty oven baked fries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zucchini cheese patties,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zucchini chocolate cake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
food_pivot.shape

(1291, 1366)

In [124]:
food_pivot.fillna(0,inplace=True)

In [125]:
food_pivot

user_id,1533,1535,2310,2312,3288,4291,4439,4470,4740,5060,...,2206637,2324285,2399085,2549237,2597942,1800054678,1802849661,2000431901,2000498330,2001102678
name,,,,,,,,,,,,,,,,,,,,,
1 pan fudge cake,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2 handed kitchen sink tomato sandwich,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20 minute creamy pesto chicken and bow ties,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24k carrots,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 minute spicy garlic shrimp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zesty oven baked fries,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zucchini cheese patties,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zucchini chocolate cake,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [126]:
#Our dataset is too sparse, but converting it into a csr_matrix will fix
from scipy.sparse import csr_matrix

In [128]:
food_sparse= csr_matrix(food_pivot)

In [129]:
food_sparse

<1291x1366 sparse matrix of type '<class 'numpy.float64'>'
	with 55979 stored elements in Compressed Sparse Row format>

In [131]:
##Now for the reccomendation model

model = NearestNeighbors(algorithm='brute')

In [132]:
model.fit(food_sparse)

NearestNeighbors(algorithm='brute')

In [140]:
#testing model
distance, suggestion = model.kneighbors(food_pivot.iloc[67].values.reshape(1,-1), n_neighbors=8)

In [141]:
distance

array([[ 0.        , 40.17461885, 40.28647416, 40.45985665, 40.48456496,
        40.75536774, 40.8900966 , 40.90232267]])

In [142]:
suggestion

array([[  67,  752, 1238,    6,  715,  243,  192,  227]])

In [143]:
for i in range(len(suggestion)):
    print(food_pivot.index[suggestion[i]])

Index(['baja black beans  corn and rice',
       'marinated chickpea and artichoke salad with feta',
       'very low fat black bean and corn salad',
       '4th of july grilled salmon  or chicken', 'lemon chicken rice soup',
       'chicken tamale casserole   cooking light', 'caramel apple dip',
       'chicken delicious'],
      dtype='object', name='name')


In [144]:
food_pivot.index

Index(['1 pan fudge cake', '2  handed kitchen sink tomato sandwich',
       '20 minute creamy pesto chicken and bow ties', '24k carrots',
       '4 minute spicy garlic shrimp', '40 minute hamburger buns',
       '4th of july grilled salmon  or chicken',
       '5 minute individual potato gratins by melissa d arabian',
       '55 house salad', 'absolute best ever lasagna',
       ...
       'yummy yogurt marinated chicken', 'zero fat soup',
       'zesty grilled pork chops', 'zesty italian crescent casserole',
       'zesty low fat chicken breasts', 'zesty oven baked fries',
       'zucchini   cheese patties', 'zucchini chocolate cake',
       'zucchini lasagna  lasagne    low carb',
       'zucchini or yellow squash casserole'],
      dtype='object', name='name', length=1291)

In [145]:
food_name = food_pivot.index

In [146]:
#Saves so we can use this in a web app
import pickle
pickle.dump(model, open('artifacts/model.pkl', 'wb'))
pickle.dump(food_name, open('artifacts/food_name.pkl', 'wb'))
pickle.dump(final_rating, open('artifacts/final_rating.pkl', 'wb'))
pickle.dump(food_pivot, open('artifacts/food_pivot.pkl', 'wb'))

In [157]:
final_rating.sample(1)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,user_id,recipe_id,date,rating,review,num_of_rating
298926,melt in your mouth pumpkin bread,14187,65,22015,2001-11-13,"['weeknight', 'time-to-make', 'course', 'cuisi...","[307.7, 24.0, 106.0, 12.0, 5.0, 12.0, 13.0]",3,"['mix oil , eggs and pumpkin', 'combine remain...",i got this recipe out of a church cookbook man...,"['oil', 'eggs', 'pumpkin', 'flour', 'sugar', '...",9,20754,14187,2001-11-23,5,This bread is awesome! It really does melt in...,28


In [184]:
#This allows us to use a name to initiate the model
def recommend_food(food_name):
    food_id = np.where(food_pivot.index==food_name)[0][0]
    distance, suggestion = model.kneighbors(food_pivot.iloc[food_id,:].values.reshape(1,-1), n_neighbors=8)
    
    for i in range(len(suggestion)):
  
        food = food_pivot.index[suggestion[i]]
        for j in food:
            print (j)

In [187]:
foodTitle='honey roasted squash rings'
recommend_food(foodTitle)

honey roasted squash rings
munchies  lentils
ruhrei     mennonite scrambled eggs
roasted root vegetables
italian lentil and barley soup
raspberry pastry roll ups
kale with caramelized onions and garlic
scrambled egg muffins


In [190]:
import anvil.server

anvil.server.connect("server_6WVWRYKNCD3DRREU6QN4RJKY-ZOB2CQ2L52GE7PAY")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER
